In [5]:
import pandas as pd
import json
from mstrio.connection import Connection
from mstrio.api import reports,change_journal,objects 
from mstrio.project_objects.report import Report
from mstrio.utils import parser
from datetime import date, timedelta,datetime
from time import sleep
from mstrio.project_objects.datasets  import super_cube
from mstrio.types import ObjectTypes, ObjectSubTypes


user_path="..\\config\\user_d.json"
with open(user_path, 'r') as file:
    user_d = json.load(file)


project_id="B7CA92F04B9FAE8D941C3E9B7E0CD754"
#project_id="40DA7A1549651FD60A9D39AAB7EC77B0"
demo_report_id="C4FF6CF34933EF4B3B1D798D02D4FB36"

conn_params =  user_d["conn_params"]
conn_params["project_id"]=project_id
conn = Connection(**conn_params)
conn.headers['Content-type'] = "application/json"

load_d={"base_url":conn.base_url,"project_name":conn.project_name,"username":conn.username}


Connection to MicroStrategy Intelligence Server has been established.


In [2]:
def get_not_packatable_type_d():
    #the change logs are full of helper objects like columns
    return { 21: "ATTRIBUTE_FORM",    63: "BLOB",    81: "CALENDAR",    24: "CATALOG",26:"COLUMN" ,   25: "CATALOG_DEFINITION",
    66: "CHANGE_JOURNAL",    26: "COLUMN",    36: "CONFIGURATION",    77: "CONTENT_BUNDLE",    57: "DBMS",
    53: "DBTABLE",    17: "FACT_GROUP",    74: "FEATURE_FLAG",    23: "FORMAT",    11: "FUNCTION",    82: "IAM",
    85: "INTERFACE_LANGUAGE",  83: "KPI_WATCHER",    70: "LAYER",    52: "LINK",    45: "LOCALE",    20: "MONITOR",
    65: "OBJECT_TAG",    42: "PACKAGE_DEFINITION",    71: "PALETTE",    32: "PROJECT",    28: "PROPERTY_SET",
    69: "RECONCILIATION",    37: "REQUEST",    19: "RESOLUTION",    80: "RUNTIME",    22: "SCHEMA",  39: "SEARCH",
    33: "SERVER_DEFINITION",    68: "SHORTCUT_TARGET",    18: "SHORTCUT_TYPE",    9: "SUBSCRIPTION_DEVICE",
    35: "SUBSCRIPTION_TRANSMITTER",    72: "THRESHOLDS",    79: "TIMEZONE"
         }

def upload_cube_mult_table(conn, mtdi_id=None, tbl_upd_dict=None,
                           cube_name=None, folder_id=None, force=False):

    if mtdi_id ==None:
        ds = super_cube.SuperCube(connection=conn, name=cube_name)
        for t in tbl_upd_dict:
            ds.add_table(name=t["tbl_name"],
                         data_frame=t["df"],
                         update_policy=t["update_policy"])
        ds.create(folder_id=folder_id,force=force)
    else:
        ds = super_cube.SuperCube(connection=conn, id=mtdi_id)
        for t in tbl_upd_dict:
            ds.add_table(name=t["tbl_name"],
                         data_frame=t["df"],
                         update_policy=t["update_policy"])
        ds.update()

    return ds.id

def trans_obj_type(rest_api_type):
    #in the change logs, we do not have numerical object types
    #for most object types we find the numerical value in the
    #module types.py of mstrio-py. Unfortunally not for all
    rest_api_type=rest_api_type.upper().strip()
    type_val=0
    try:
        type_val=ObjectTypes[rest_api_type].value
    except:
        
        if rest_api_type=="COMMAND_MANAGER_SCRIPT":
            type_val=76

        elif rest_api_type=="DB_TABLE":
            type_val=56
        elif rest_api_type=="TYPE_SHORTCUT":
            type_val=67
        else:
            print("Type not found")
            print(rest_api_type)
    return type_val

def check_obj_packatable(conn, obj_id,type_val,change_d,change_d_l):
    #the change logs are full of helper objects like columns
    #further the change logs contain deleted objects
    #here we ensure, that we only put packageable objets
    #into the cube
    type_val=change_d["object_type_val"]
    if type_val not in list(get_not_packatable_type_d().keys()):
        obj_ck_resp=conn.get(f'{conn.base_url}/api/objects/{obj_id}?type={type_val}&comments=false' )
        #print(dir(obj_ck_resp))
        if obj_ck_resp.status_code==200:
            obj_ck_d=obj_ck_resp.json()
            path_s=""
            change_d["project_name"]=obj_ck_resp.json()["ancestors"][0]
            for fol in obj_ck_resp.json()["ancestors"]:
                path_s= path_s + "\\" + fol["name"]
            change_d["folder"]=path_s
            if obj_ck_d["ancestors"][0]["name"]!= "Managed Objects":
                change_d_l.append(change_d.copy())
    return change_d_l

def prep_load_cube(conn, change_d_l,cube_id,cube_name,cube_folder_id):
    #to prepare migration package a time dimension is very usefull
    # in the last step we load the dataframe to MSTR
    change_obj_log_df=pd.DataFrame(change_d_l)
    #change_obj_log_df['timestamp'] = pd.to_datetime(change_obj_log_df['timestamp'])
    change_obj_log_df['mod_date'] =pd.to_datetime(change_obj_log_df["timestamp"]).dt.date
    change_obj_log_df['mod_week'] =pd.to_datetime(change_obj_log_df["timestamp"]).dt.isocalendar().week
    change_obj_log_df['mod_month'] =pd.to_datetime(change_obj_log_df["timestamp"]).dt.month
    change_obj_log_df['mod_quarter'] =pd.to_datetime(change_obj_log_df["timestamp"]).dt.quarter
    change_obj_log_df['mod_year'] =pd.to_datetime(change_obj_log_df["timestamp"]).dt.year
    #change_obj_log_df['timestamp'] = pd.to_datetime(change_obj_log_df['timestamp'].astype(str)
    
    change_obj_log_d_l=change_obj_log_df.to_dict(orient="records")
    change_obj_log_df=pd.DataFrame(change_obj_log_d_l)
    change_obj_log_df=change_obj_log_df.astype('str')
    change_obj_log_d_l=[{"df":change_obj_log_df,"tbl_name":"change_obj_log_df", "update_policy":"Replace" }]

    cube_id=upload_cube_mult_table(conn=conn, mtdi_id=cube_id, tbl_upd_dict=change_obj_log_d_l,
                              cube_name=cube_name, folder_id=cube_folder_id)

    return cube_id

def load_change_log_to_cube(conn,project_id,start_date_str,cube_id,cube_name,cube_folder_id, limit=100):
    #this is the main script to read out change logs
    #first we create a search Instance, than we loop through the lines
    conn.select_project(project_id)
    body_d=  {"affectedProjects": [project_id],   "beginTime": start_date_str}
    search_id=change_journal.create_change_journal_search_instance(connection=conn,body=body_d).json()["searchId"]
    change_d_l=[]
    no_rows_fg=0
    offset=0
    while no_rows_fg==0:
        res=change_journal.get_change_journal_search_results(connection=conn,search_id=search_id,offset=offset,limit=limit).json()
        if len(res["changeJournalEntries"])==0:
            no_rows_fg=1
            
        offset+=limit
        for c in res["changeJournalEntries"]:
            change_d={}
            log_entry_type=None
            change_d["projectId"]=c["projectId"]
            change_d["machine"]=c["machine"]
            change_d["timestamp"]=c["timestamp"]
            change_d["sessionId"]=c["sessionId"]
            change_d["transaction_id"]=c["transaction"]["id"]
            change_d["transaction_type"]=c["transaction"]["type"]
            change_d["transaction_source "]=c["transaction"]["source"]
            change_d["user_id"]=c["user"]["id"]
            change_d["user_name"]=c["user"]["name"]
            if "changedObjects" in c.keys():
                for trans in c["changedObjects"]:
                    change_d["changeType"]=trans["changeType"]
                    change_d["object_id"]=trans["object"]["id"]
                    change_d["object_name"]=trans["object"]["name"]
                    change_d["object_type"]=trans["object"]["type"]
                    
                    if "userComments" in trans.keys():
                        change_d["userComments"]=trans["userComments"]
                    else:
                        change_d["userComments"]=""
                    
                    change_d["object_type_val"]=trans_obj_type(rest_api_type=trans["object"]["type"])

                    change_d_l= check_obj_packatable(conn=conn, obj_id=trans["object"]["id"],
                                                     type_val=change_d["object_type_val"],
                                                     change_d=change_d,change_d_l=change_d_l)

                    
    cube_id=prep_load_cube(conn, change_d_l,cube_id,cube_name,cube_folder_id)

    return cube_id


In [3]:
start_date_str="2025-03-28"
cube_id="CFBA8E864BA87CE0615C4CADAE0F731F"
#cube_id=None
cube_name="change_obj_log_1"
cube_folder_id="CF2049E94A0000A16532D39C6D783F1D"
change_obj_log_df=load_change_log_to_cube(conn,project_id,start_date_str,cube_id,cube_name,cube_folder_id, limit=100)
change_obj_log_df

SuperCube object named: 'change_obj_log' with ID: 'CFBA8E864BA87CE0615C4CADAE0F731F'


Uploading 1/1: 100%|██████████| 1/1 [00:00<00:00, 14.81it/s, rows=297]


Super cube 'change_obj_log' published successfully.


'CFBA8E864BA87CE0615C4CADAE0F731F'

In [ ]:
offset=0
limit=100
body_d=  {"affectedProjects": [project_id],   "beginTime": start_date_str}
search_id=change_journal.create_change_journal_search_instance(connection=conn,body=body_d).json()["searchId"]
res=change_journal.get_change_journal_search_results(connection=conn,search_id=search_id,offset=offset,limit=limit).json()
res

In [ ]:
res=change_journal.get_change_journal_search_results(connection=conn,search_id=search_id,offset=offset,limit=limit).json()
res

In [ ]:
excel_file="D:/shared_drive/MSTR_Migrations/WS_Migrations/daniel_obj_mig_log.xlsx"

df = pd.read_excel(excel_file)
df

In [7]:
from mstrio.object_management.migration import (
    PackageType,
    Migration,
    PackageConfig,
    PackageSettings
)

from mstrio.object_management.migration.package import (
    Action,
    ImportStatus,
    PackageStatus,
    ValidationStatus,
)


def bld_mig_list_from_df(df,action="FORCE_REPLACE",include_dependents=False):
    mig_obj_d_l=[]
    for row,obj in df.iterrows():
        obj_d={}
        obj_d["id"] = obj["object_id"]
        obj_d["type"] = obj["object_type_val"]
        obj_d["action"] = action
        obj_d["include_dependents"] = include_dependents
        mig_obj_d_l.append(obj_d.copy())
    return mig_obj_d_l

def create_validate_obj_pk(conn, mig_obj_d_l,package_settings,package_content_info,package_name):
    package_content_info=bld_mig_list_from_df(df, action= "FORCE_REPLACE",include_dependents= True)
    package_config = PackageConfig(
        package_settings,  package_content_info # [ package_content_from_object]
    )
    
    obj_mig = Migration.create_object_migration(
        connection=conn,
        toc_view=package_config,
        name=package_name,
        project_id=conn.project_id,
    )
    
    while obj_mig.package_info.status == PackageStatus.CREATING:
        sleep(2)
        obj_mig.fetch()
    return obj_mig

package_settings = PackageSettings(
    Action.FORCE_REPLACE,
    PackageSettings.UpdateSchema.RECAL_TABLE_LOGICAL_SIZE,
    PackageSettings.AclOnReplacingObjects.REPLACE,
    PackageSettings.AclOnNewObjects.KEEP_ACL_AS_SOURCE_OBJECT,
)
excel_file="D:/shared_drive/MSTR_Migrations/WS_Migrations/daniel_obj_mig_log.xlsx"
package_name="WS_DEMO"
df = pd.read_excel(excel_file)
package_content_info=bld_mig_list_from_df(df)
obj_mig=create_validate_obj_pk(conn, package_content_info,package_settings,package_content_info,package_name)
obj_mig

Successfully started creation of migration object with ID: '28745166AFDF4681997D98AB1D9C88DC:D0002ECD6CAA4EE3998221FDE0057311'


Migration(connection, id='28745166AFDF4681997D98AB1D9C88DC:D0002ECD6CAA4EE3998221FDE0057311', name='WS_DEMO')

In [ ]:
obj_id="150349F04560BBA2592D019726DF77DD"
type_val="4"
obj_ck_resp=conn.get(f'{conn.base_url}/api/objects/{obj_id}?type={type_val}&comments=false' )
obj_ck_resp

In [ ]:
path_s=""
for fol in obj_ck_resp.json()["ancestors"]:
    path_s= path_s + "\\" + fol["name"]
    #path_s+=r"\\"
print(path_s)

In [ ]:
s = r"folder" + '\\' + r"file.txt"
print(s)


PackageInfo(name='WS_DEMO', creator=User(connection, name='Administrator', id='54F3D26011D2896560009A8E67019608'), creation_date=datetime.datetime(2025, 4, 29, 13, 1, 30, tzinfo=datetime.timezone.utc), last_updated_date=datetime.datetime(2025, 4, 29, 13, 1, 35, tzinfo=datetime.timezone.utc), type=<PackageType.OBJECT: 'project'>, environment={'id': 'http://85.214.60.83:8080/MicroStrategyLibrary/', 'name': 'http://85.214.60.83:8080/MicroStrategyLibrary/'}, id='28745166AFDF4681997D98AB1D9C88DC', replicated=False, storage=PackageStorage(size=371682, path='mstr/shared/migrations/packages/2025-04-29/14F89D6C7B7847778ED7E59CE2206470.mmp'), project=Project(connection, name='MicroStrategy Tutorial', id='B7CA92F04B9FAE8D941C3E9B7E0CD754'), status=PackageStatus.CREATED, message='', warnings=None, progress=100.0, deleted=False, existing=True, toc_view=None, purpose=MigrationPurpose.OBJECT_MIGRATION, tree_view=None, certification=None)
